*This second part of the tutorial needs to be run on your local laptop or desktop machine. First, make sure you are running Jupyter on your laptop or desktop. Then, click the Download button above and save the .ipynb file to your Jupyter startup folder. After the download completes, the notebook will show up in the Jupyter Dashboard. Open the downloaded notebook from the Dashboard and carry on with the tutorial steps in that copy of the notebook. (You can run this tutorial in Azure, but the final steps will fail with a Timeout Error because the Raspberry Pi device cannot be reached on the network.)*

The first few steps of this tutorial are repeated from Part 1. Just like the beginning of that tutorial, we choose a pre-trained model and download it (this time to your laptop or desktop machine).

In [ ]:
!conda config --prepend channels conda-forge --prepend channels microsoft-ell
!conda install -y ell

In [ ]:
from ell.util.gallery import ModelGallery
gallery = ModelGallery()
gallery.choose_model()

In [ ]:
pretrained_model = gallery.model
pretrained_model.download('getting-started', rename='model')

If you like, you can repeat the last steps of Part 1 to build the model on the host machine and try it to make predictions on that machine. Here, let's look at building the model for the Rapsberry Pi and making predictions on that device.

## Compile the model for the Raspberry Pi

We are ready to cross-compile the model for deployment on the Raspberry Pi. Use the `compile` method and specify the target platform as `PI3`. This tells the ELL compiler to generate machine code for the Raspberry Pi's ARM Cortex A53 processor. The `compile` method creates a new directory named `pi3`, which contains a CMake project that can be used to build the desired Python module. This time, we need to build this project on the Raspberry Pi.

In [ ]:
import ell.platform
pretrained_model.compile(ell.platform.PI3)

## Write code to invoke the model on the Raspberry Pi

Deploy the model to the Raspberry Pi and try it out by using the Jupyter "magic" called `%%rpi`. This magic requires several arguments: The name of the user on the Raspberry Pi (typically "pi"), the Pi's IP address on the local network, the directory where the model will be downloaded, and the Python variable that refers to our model. **Change the `ip` and `user` arguments to your Raspberry Pi's IP address and your user name before running the code in the cell below.**

Running this cell does several things. First, it prompts for the user password on the Pi. (This password is remembered during a notebook session, so it only needs to be entered once.) Second, it downloads the code from the local `pi3` directory to the directory on the Pi specified by `rpipath`. Next, it builds the CMake project on the Pi. (Again, this is only necessary once per model.) Finally, it runs the cell's Python code on the Pi. Any output that the Python code prints will be shown in the notebook. Also, because the Python code could contain a loop, we provide a button to allow you to stop the code running on the Pi.

Just like in Part 1, start by downloading a known image and run the model on it.

In [ ]:
%%rpi --user=pi --ip=157.54.152.78 --rpipath=/home/pi/mymodel --model=pretrained_model

import cv2
import numpy as np
import urllib.request
import tutorialHelpers as helpers
import sys
sys.path.append('build')
import model

urllib.request.urlretrieve('https://microsoft.github.io/ELL/tutorials/shared/coffeemug.jpg', 'coffeemug.jpg')
image = cv2.imread("coffeemug.jpg")

inputShape = model.get_default_input_shape()
outputShape = model.get_default_output_shape()
predictions = model.FloatVector(outputShape.Size())

categories = [line.strip('\n') for line in open('categories.txt', 'r').readlines()]

input = helpers.prepare_image_for_model(image, inputShape.columns, inputShape.rows)
model.predict(input, predictions)
top5 = helpers.get_top_n_predictions(predictions, 5)
text = "".join(["(" + str(int(conf * 100)) + "%) " + categories[index] for (index, conf) in top5])
print('TOP 5:')
print(text)

After seeing it work on a static image, run the cell below to classify an object in a web cam image. This code works like a camera: when you run the cell, it causes the Pi to take a web cam image and runs that image through the model.

In [ ]:
%%rpi --user=pi --ip=157.54.152.78 --rpipath=/home/pi/mymodel --model=pretrained_model

import cv2
import numpy as np
import urllib.request
import tutorialHelpers as helpers
import sys
sys.path.append('build')
import model

def get_image_from_camera(camera):
    if camera is not None:
        # if predictor is too slow frames get buffered, this is designed to flush that buffer
        ret, frame = camera.read()
        if (not ret):
            raise Exception('your capture device is not returning images')
        return frame
    return None

camera = cv2.VideoCapture(0)
image = get_image_from_camera(camera)

inputShape = model.get_default_input_shape()
outputShape = model.get_default_output_shape()
predictions = model.FloatVector(outputShape.Size())

categories = [line.strip('\n') for line in open('categories.txt', 'r').readlines()]

input = helpers.prepare_image_for_model(image, inputShape.columns, inputShape.rows)
model.predict(input, predictions)
top5 = helpers.get_top_n_predictions(predictions, 5)
text = "".join(["(" + str(int(conf * 100)) + "%) " + categories[index] for (index, conf) in top5])
print('TOP 5:')
print(text)

If you like, you can also add a loop around this code to classify one web cam image after another. To stop the code from running on the Pi, click the Stop button that appears.